import pandas as pd

In [1]:
import pandas as pd

In [3]:
evohia = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\Evohia.csv', error_bad_lines=False);
semeli = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Semeli Hotel.csv', error_bad_lines=False);
landmark = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\The Landmark Nicosia.csv', error_bad_lines=False);

tripadv_review = pd.concat([evohia, semeli, landmark])
tripadv_review

,r_points,r_text,place
0,30,I don't know what's so elegant about this plac...,Evohia
1,40,"Over the years, this place doesn't change. It ...",Evohia
2,50,Close to center a very friendly place with a m...,Evohia
3,30,the buffet did not have cakes although they ar...,Evohia
4,50,Very good buffet and friendly staff. Offers he...,Evohia
...,...,...,...
1098,50,The Nicosia Hilton is beautiful - there are pa...,The Landmark Nicosia
1099,50,Very good first-class hotel in Nicosia. We sta...,The Landmark Nicosia
1100,20,We stayed a couple of days in Nicosia for tour...,The Landmark Nicosia
1101,50,"Stayed here for a week in July 2004, and 2 wee...",The Landmark Nicosia


In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [7]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0       [know, elegant, place, good, interior, design,...
1       [years, place, change, reaches, high, standard...
2       [close, center, friendly, place, menu, taste, ...
3          [buffet, cakes, advertised, pics, extra, cake]
4       [good, buffet, friendly, staff, offers, health...
                              ...                        
1098    [nicosia, hilton, beautiful, palm, trees, beau...
1099    [good, class, hotel, nicosia, stayedin, execut...
1100    [stayed, couple, days, nicosia, tourism, stay,...
1101    [stayed, week, july, weeks, october, agree, pr...
1102    [stayed, business, recommend, visiting, nicosi...
Name: r_text, Length: 1382, dtype: object

In [8]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]]


In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=1500,
                                           passes=15,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 225,
                                           eval_every = None)

In [10]:
from pprint import pprint

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"food" + 0.005*"hilton" + 0.005*"people" + 0.005*"hotel" + '
  '0.005*"guests" + 0.004*"need" + 0.004*"minus" + 0.004*"lunch" + '
  '0.004*"including" + 0.003*"reasonable"'),
 (1,
  '0.025*"hotel" + 0.022*"room" + 0.020*"good" + 0.013*"staff" + '
  '0.013*"breakfast" + 0.008*"nice" + 0.008*"food" + 0.008*"hilton" + '
  '0.007*"clean" + 0.007*"great"'),
 (2,
  '0.011*"hotel" + 0.010*"buffet" + 0.007*"nice" + 0.007*"room" + '
  '0.006*"service" + 0.006*"hilton" + 0.005*"days" + 0.005*"choice" + '
  '0.005*"rooms" + 0.005*"executive"'),
 (3,
  '0.015*"staff" + 0.013*"hotel" + 0.012*"room" + 0.008*"breakfast" + '
  '0.008*"pool" + 0.006*"time" + 0.005*"business" + 0.004*"nice" + '
  '0.004*"euros" + 0.004*"guests"'),
 (4,
  '0.032*"hotel" + 0.023*"staff" + 0.017*"good" + 0.015*"stay" + 0.014*"pool" '
  '+ 0.013*"room" + 0.013*"rooms" + 0.012*"service" + 0.012*"breakfast" + '
  '0.010*"nice"'),
 (5,
  '0.032*"hotel" + 0.025*"staff" + 0.023*"good" + 0.018*"hilton" + '
  '0.015*

In [12]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.084994 -0.005787       1        1  33.067303
5      0.109532 -0.016610       2        1  24.718058
4      0.087819 -0.010847       3        1  12.872809
9      0.050482 -0.062438       4        1   8.373712
1      0.046459  0.031945       5        1   7.446052
7      0.040487  0.025320       6        1   5.660962
8     -0.052723  0.137468       7        1   2.861927
3     -0.065905 -0.032163       8        1   2.563583
2     -0.128352 -0.056770       9        1   1.483844
0     -0.172794 -0.010116      10        1   0.951756, topic_info=           Term         Freq        Total Category  logprob  loglift
174       hotel  1837.000000  1837.000000  Default  30.0000  30.0000
838        room   800.000000   800.000000  Default  29.0000  29.0000
6          good   971.000000   971.000000  Default  28.0000  28.0000
118       staff   987.000000   987.000000  Default  27.0000  27.0000
5          food   430.000000   430.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
840        stay     1.457162   513.494324  Topic10  -5.9147  -1.2101
12   restaurant     1.357338   275.385071  Topic10  -5.9856  -0.6580
169      better     1.232045    76.576401  Topic10  -6.0825   0.5250
6          good     1.228946   971.789551  Topic10  -6.0850  -2.0184
476      cyprus     1.181363   365.014954  Topic10  -6.1245  -1.0787

[854 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
957       1  0.493398  access
957       2  0.156443  access
957       3  0.036102  access
957       4  0.024068  access
957       5  0.084239  access
...     ...       ...     ...
5849      9  0.757646  لايوجد
5850      9  0.757646    لدول
5851      9  0.757646    مللا
5852      9  0.757646   وسائل
5853      9  0.757646   وسيلة

[1959 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 5, 10, 2, 8, 9, 4, 3, 1])